In [1]:
import pandas as pd

## 设置时间片大小

In [2]:
timestamp = 20

## 1. 读取和合并二级意图识别后的数据

In [3]:
df_user_1 = pd.read_csv('Temporary Data/1_LML_predicted_data.csv')
df_user_2 = pd.read_csv('Temporary Data/2_ZRS_predicted_data.csv')
df_user_3 = pd.read_csv('Temporary Data/3_MM_predicted_data.csv')
df_user_4 = pd.read_csv('Temporary Data/4_JMX_predicted_data.csv')
df_user_5 = pd.read_csv('Temporary Data/5_ZJY_predicted_data.csv')
df_user_6 = pd.read_csv('Temporary Data/6_WZX_predicted_data.csv')
df_user_7 = pd.read_csv('Temporary Data/7_WBJ_predicted_data.csv')
df_user_8 = pd.read_csv('Temporary Data/8_YZY_predicted_data.csv')
df_user_9 = pd.read_csv('Temporary Data/9_QT_predicted_data.csv')
df_user_10 = pd.read_csv('Temporary Data/10_ZZW_predicted_data.csv')


my_data = pd.concat([df_user_1, df_user_2, df_user_3, df_user_4, df_user_5, 
                     df_user_6, df_user_7, df_user_8, df_user_9, df_user_10], ignore_index=True)
print(len(df_user_1), len(df_user_2), len(df_user_3), len(df_user_4), len(df_user_5), 
      len(df_user_6), len(df_user_7), len(df_user_8), len(df_user_9), len(df_user_10))
print(len(my_data))
my_data.head()

19424 25165 23949 19207 23981 27731 21547 23693 23364 25025
233086


,Unnamed: 0,User,ID,Time,Hotas_Axis_X,Hotas_Axis_Y,Hotas_Axis_Z,Hotas_Btn_Create_Fleet,Hotas_Btn_Select_UAV1,Hotas_Btn_Select_UAV2,...,Target_Area_Pos_X,Target_Area_Pos_Y,Trapped_People_State,Trapped_People_Pos_X,Trapped_People_Pos_Y,Gaze point X,Gaze point Y,AOI,Primary Intent,Secondary Intent
0,0,1,5,1900-01-01 14:46:17.924,0.0,0.0,-0.421107,False,False,False,...,-1.0,-1.0,False,-1.0,-1.0,2031.0,1247.0,0,Intent_0,NaN
1,1,1,5,1900-01-01 14:46:17.949,0.0,0.0,-0.421107,False,False,False,...,-1.0,-1.0,False,-1.0,-1.0,2232.0,1230.0,4,Intent_0,NaN
2,2,1,5,1900-01-01 14:46:17.983,0.0,0.0,-0.421107,False,False,False,...,-1.0,-1.0,False,-1.0,-1.0,2241.0,1210.0,4,Intent_0,NaN
3,3,1,5,1900-01-01 14:46:18.011,0.0,0.0,-0.421107,False,False,False,...,-1.0,-1.0,False,-1.0,-1.0,2239.0,1223.0,4,Intent_1,NaN
4,4,1,5,1900-01-01 14:46:18.037,0.0,0.0,-0.421107,False,False,False,...,-1.0,-1.0,False,-1.0,-1.0,2252.0,1209.0,4,Intent_1,NaN


## 2. 整理为DBN输入格式

In [4]:
# 将一级意图转换为数字序号
def process_primary_intention(df):
    df['Primary_Intention'] = None
    for index, row in df.iterrows():
        if row['Primary Intent'] == 'Intent_0':
            df.at[index, 'Primary_Intention'] = 0
        if row['Primary Intent'] == 'Intent_1':
            df.at[index, 'Primary_Intention'] = 1
        if row['Primary Intent'] == 'Intent_2':
            df.at[index, 'Primary_Intention'] = 2
        if row['Primary Intent'] == 'Intent_3':
            df.at[index, 'Primary_Intention'] = 3
        if row['Primary Intent'] == 'Intent_4':
            df.at[index, 'Primary_Intention'] = 4
        if row['Primary Intent'] == 'Intent_5':
            df.at[index, 'Primary_Intention'] = 5
    df.drop(columns=['Primary Intent'], inplace=True)
    return df


# 将HOTAS按键转换为数字信号
def process_hotas_button(df):
    df['Hotas_Button'] = None
    for index, row in df.iterrows():
        if row['Hotas_Btn_Create_Fleet']:
            df.at[index, 'Hotas_Button'] = 1
        elif row['Hotas_Btn_Select_UAV1']:
            df.at[index, 'Hotas_Button'] = 2
        elif row['Hotas_Btn_Select_UAV2']:
            df.at[index, 'Hotas_Button'] = 3
        elif row['Hotas_Btn_Select_UAV3']:
            df.at[index, 'Hotas_Button'] = 4
        elif row['Hotas_Btn_Select_UAV4']:
            df.at[index, 'Hotas_Button'] = 5
        elif row['Hotas_Btn_Assign_Up']:
            df.at[index, 'Hotas_Button'] = 6
        elif row['Hotas_Btn_Assign_Down']:
            df.at[index, 'Hotas_Button'] = 7
        elif row['Hotas_Btn_Assign_Left']:
            df.at[index, 'Hotas_Button'] = 8
        elif row['Hotas_Btn_Assign_Right']:
            df.at[index, 'Hotas_Button'] = 9
        elif row['Hotas_Btn_Change_Mode']:
            df.at[index, 'Hotas_Button'] = 10
        elif row['Hotas_Btn_Open_UAV_Sensor']:
            df.at[index, 'Hotas_Button'] = 11
        elif row['Hotas_Btn_Open_Helicopter_Sensor']:
            df.at[index, 'Hotas_Button'] = 12
        elif row['Hotas_Btn_Select_Normal']:
            df.at[index, 'Hotas_Button'] = 13
        elif row['Hotas_Btn_Select_Fire']:
            df.at[index, 'Hotas_Button'] = 14
        elif row['Hotas_Btn_Select_Person']:
            df.at[index, 'Hotas_Button'] = 15
        elif row['Hotas_Btn_Submit_Result']:
            df.at[index, 'Hotas_Button'] = 16
        elif row['Hotas_Btn_Area_Assign']:
            df.at[index, 'Hotas_Button'] = 17
        else:
            df.at[index, 'Hotas_Button'] = 0

    df.drop(columns=['Hotas_Btn_Create_Fleet', 'Hotas_Btn_Select_UAV1', 'Hotas_Btn_Select_UAV2',
                     'Hotas_Btn_Select_UAV3', 'Hotas_Btn_Select_UAV4', 'Hotas_Btn_Assign_Up',
                     'Hotas_Btn_Assign_Down', 'Hotas_Btn_Assign_Left', 'Hotas_Btn_Assign_Right',
                     'Hotas_Btn_Change_Mode', 'Hotas_Btn_Open_UAV_Sensor', 'Hotas_Btn_Open_Helicopter_Sensor',
                     'Hotas_Btn_Select_Normal', 'Hotas_Btn_Select_Fire', 'Hotas_Btn_Select_Person',
                     'Hotas_Btn_Submit_Result', 'Hotas_Btn_Area_Assign'],
            inplace=True)
    return df


# 将本机状态-选择无人机序号修正为0,1,2,3,4
def process_select_UAV_num(df):
    df['Select_UAV_Num'] = None
    for index, row in df.iterrows():
        if row['UAV_Selected_Now'] == -1:
            df.at[index, 'Select_UAV_Num'] = 0
        else:
            df.at[index, 'Select_UAV_Num'] = row['UAV_Selected_Now']
    df.drop(columns=['UAV_Selected_Now'], inplace=True)
    return df


# 将所有True或False的数据对应转换为0或1
def process_bool_to_int(df):
    df['Flight_Sensor'] = df['Flight_Sensor'].replace({True: 0, False: 1})
    df['UAV_Sensor_State'] = df['UAV_Sensor_State'].replace({True: 0, False: 1})
    df['Task_List_State'] = df['Task_List_State'].replace({True: 0, False: 1})
    df['Target_Area_State'] = df['Target_Area_State'].replace({True: 0, False: 1})
    df['Trapped_People_State'] = df['Trapped_People_State'].replace({True: 0, False: 1})
    return df


# 准备BN数据
def prepare_data_for_DBN(df):
    df = process_primary_intention(df)
    df = process_hotas_button(df)
    df = process_select_UAV_num(df)
    df.drop(columns=['User', 'Time', 'Hotas_Axis_X', 'Hotas_Axis_Y', 'Hotas_Axis_Z',
                     'Flight_Pos_X', 'Flight_Pos_Y', 'Flight_Speed', 'Flight_Height', 'Flight_Heading',
                     'UAV1_Pos_X', 'UAV1_Pos_Y', 'UAV2_Pos_X', 'UAV2_Pos_Y', 'UAV3_Pos_X', 'UAV3_Pos_Y',
                     'UAV4_Pos_X', 'UAV4_Pos_Y', 'Target_Area_Pos_X', 'Target_Area_Pos_Y',
                     'Trapped_People_Pos_X', 'Trapped_People_Pos_Y', 'Gaze point X', 'Gaze point Y'],
            inplace=True)
    # 将所有True或False的数据对应转换为0或1
    df = process_bool_to_int(df)
    return df


# 将整理好的数据格式切换为：Feature, Feature_1, Feature_2……的时间片格式
def slip_DBN_timestamps(df, time_steps):
    # 初始化新DataFrame，用于存储结果
    result = df.copy()
    for i in range(1, time_steps):
        # 创建一个新的DataFrame并右移i步长
        shifted_df = df.shift(-i)
        # 重命名列
        shifted_df.columns = [f'{col}_{i}' for col in df.columns]
        # 将新DataFrame拼接到result右边
        result = pd.concat([result, shifted_df], axis=1)

    # 删除来自不同试次即不同ID的行
    id_columns = [col for col in result.columns if col.startswith('ID')]
    mask = result[id_columns].nunique(axis=1) == 1 # 布尔掩码
    result = result[mask]

    # 删除多余的ID列
    for i in range(1, time_steps):
        result.drop(columns=[f'ID_{i}'], inplace=True)

    # 删除超出范围的行
    result = result.iloc[:-time_steps + 1]
    return result

def delete_DBN_timestamps_Intent0(df):
    intent_columns = [col for col in df.columns if col.startswith('Primary_Intention')]
    filtered_data = df[~df[intent_columns].eq(0).any(axis=1)]
    filtered_data.loc[:, intent_columns] = filtered_data.loc[:, intent_columns]-1
    return filtered_data

In [5]:
dbn_data = prepare_data_for_DBN(my_data)
dbn_data.rename(columns={'Secondary Intent': 'Secondary_Intention'}, inplace=True)
dbn_data['Secondary_Intention'] = dbn_data['Secondary_Intention'].fillna(0)

dbn_data_times = slip_DBN_timestamps(dbn_data, timestamp)
dbn_data_times = delete_DBN_timestamps_Intent0(dbn_data_times)
dbn_data_times.to_csv(f'Dataset/DBN_Test_Data_{timestamp}times.csv', index=False)

In [6]:
dbn_data.head()

,Unnamed: 0,ID,Flight_Fleet_State,Flight_Fly_Mode,Flight_Sensor,UAV1_Task_State,UAV2_Task_State,UAV3_Task_State,UAV4_Task_State,UAV_Sensor_State,Task_List_State,Target_Area_State,Trapped_People_State,AOI,Secondary_Intention,Primary_Intention,Hotas_Button,Select_UAV_Num
0,0,5,0,0,1,0,0,0,0,1,0,1,1,0,0.0,0,0,0
1,1,5,0,0,1,0,0,0,0,1,0,1,1,4,0.0,0,0,0
2,2,5,0,0,1,0,0,0,0,1,0,1,1,4,0.0,0,0,0
3,3,5,0,0,1,0,0,0,0,1,0,1,1,4,0.0,1,0,0
4,4,5,0,0,1,0,0,0,0,1,0,1,1,4,0.0,1,0,0
